In [6]:
import os, sys
sys.path.append("../../../")

from src.core.module import Module, Linear, LayerNorm
from src.core.losses import CrossEntropyWithLogits, BinaryCrossEntropyWithLogits
from src.core.optim import Standard, AdamW
from src.core.tensor import Tensor
from src.utils.lr_scheduler import LRScheduler
import numpy as np
import time
from typing import List
# from src.tokenizer.tokenizer import Tokenizer
import pandas as pd

In [16]:
class Net(Module):
    def __init__(self):
        super().__init__()
        self.fc1 = self.linear(7, 28, name="fc1")
        self.fc2 = self.linear(28, 2, name="fc2")
        self.ln = self.layer_norm(axis=-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.ln(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x
    
    def train(self, x: Tensor, y: Tensor, optimizer, num_epochs=100):
        for epoch in range(num_epochs):
            y_hat = self.forward(x)
            
            loss = CrossEntropyWithLogits(y_hat, y) * 0.1
            
            loss.backward()
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss.data * 10}")
                optimizer.step()
            
                optimizer.zero_grad()






df = pd.read_csv("../../../src/experiments/data.csv")
df = df.sample(frac=1).reset_index(drop=True)
df['Quality'] = df['Quality'].apply(lambda x: 1 if x == "Good" else 0)
X = Tensor(np.array(df.drop('Quality', axis=1).values))[:128]
y = Tensor(np.array(df['Quality'].values))[:128] 

X_test = Tensor(np.array(df.drop('Quality', axis=1).values))[128:]
y_test = Tensor(np.array(df['Quality'].values)).reshape((-1, 1))[128:]
net = Net()

net._build(X.shape)
optimizer = AdamW(net.parameters(), lr=0.001, clip_norm=100.0)

net.train(X, y, optimizer, num_epochs=2500)


Epoch 9, Loss: 0.7861328125
Epoch 19, Loss: 0.775390625
Epoch 29, Loss: 0.7626953125
Epoch 39, Loss: 0.7509765625
Epoch 49, Loss: 0.73779296875
Epoch 59, Loss: 0.72509765625
Epoch 69, Loss: 0.71240234375
Epoch 79, Loss: 0.69921875
Epoch 89, Loss: 0.6875
Epoch 99, Loss: 0.6748046875
Epoch 109, Loss: 0.6630859375
Epoch 119, Loss: 0.650390625
Epoch 129, Loss: 0.6396484375
Epoch 139, Loss: 0.6279296875
Epoch 149, Loss: 0.6162109375
Epoch 159, Loss: 0.60595703125
Epoch 169, Loss: 0.59521484375
Epoch 179, Loss: 0.5849609375
Epoch 189, Loss: 0.57470703125
Epoch 199, Loss: 0.56494140625
Epoch 209, Loss: 0.55517578125
Epoch 219, Loss: 0.5458984375
Epoch 229, Loss: 0.537109375
Epoch 239, Loss: 0.5283203125
Epoch 249, Loss: 0.51953125
Epoch 259, Loss: 0.51123046875
Epoch 269, Loss: 0.50341796875
Epoch 279, Loss: 0.495361328125
Epoch 289, Loss: 0.48779296875
Epoch 299, Loss: 0.47998046875
Epoch 309, Loss: 0.47314453125
Epoch 319, Loss: 0.466064453125
Epoch 329, Loss: 0.459228515625
Epoch 339, Loss

In [ ]:
# Epoch 9, Loss: 1.07421875
# Epoch 19, Loss: 1.060546875
# Epoch 29, Loss: 1.0439453125
# Epoch 39, Loss: 1.0283203125
# Epoch 49, Loss: 1.01171875
# Epoch 59, Loss: 0.99462890625
# Epoch 69, Loss: 0.9775390625
# Epoch 79, Loss: 0.9609375
# Epoch 89, Loss: 0.9443359375
# Epoch 99, Loss: 0.92822265625
# Epoch 109, Loss: 0.91162109375
# Epoch 119, Loss: 0.89599609375
# Epoch 129, Loss: 0.8798828125
# Epoch 139, Loss: 0.8642578125
# Epoch 149, Loss: 0.84912109375
# Epoch 159, Loss: 0.833984375
# Epoch 169, Loss: 0.81884765625
# Epoch 179, Loss: 0.8046875
# Epoch 189, Loss: 0.7900390625
# Epoch 199, Loss: 0.77587890625
# Epoch 209, Loss: 0.76171875
# Epoch 219, Loss: 0.74853515625
# Epoch 229, Loss: 0.73486328125
# Epoch 239, Loss: 0.7216796875
# Epoch 249, Loss: 0.708984375

In [101]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 28)
        self.fc4 = nn.Linear(28, 3)  # 3 classes
        self.gelu = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc4(x)
        return x  # raw logits, no sigmoid

# Load + prep data
df = pd.read_csv("../../../src/experiments/data.csv")
df = df.sample(frac=1).reset_index(drop=True)

# Map classes properly starting at 0
df['Quality'] = df['Quality'].apply(lambda x: 0 if x == "Good" else (1 if x == "Bad" else 2))  # Adjust if you have 3 classes

X = torch.tensor(df.drop('Quality', axis=1).values, dtype=torch.float32)
y = torch.tensor(df['Quality'].values, dtype=torch.long)  # LONG for CE loss!

# Train/test split
X_train, y_train = X[:128], y[:128]
X_test, y_test = X[128:], y[128:]

# Initialize model, loss, optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training loop
for epoch in range(250):
    model.train()

    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1.3029
Epoch 10, Loss: 1.1438
Epoch 20, Loss: 1.0108
Epoch 30, Loss: 0.8985
Epoch 40, Loss: 0.8017
Epoch 50, Loss: 0.7168
Epoch 60, Loss: 0.6414
Epoch 70, Loss: 0.5736
Epoch 80, Loss: 0.5124
Epoch 90, Loss: 0.4572
Epoch 100, Loss: 0.4080
Epoch 110, Loss: 0.3646
Epoch 120, Loss: 0.3270
Epoch 130, Loss: 0.2947
Epoch 140, Loss: 0.2674
Epoch 150, Loss: 0.2442
Epoch 160, Loss: 0.2246
Epoch 170, Loss: 0.2080
Epoch 180, Loss: 0.1938
Epoch 190, Loss: 0.1817
Epoch 200, Loss: 0.1711
Epoch 210, Loss: 0.1617
Epoch 220, Loss: 0.1534
Epoch 230, Loss: 0.1459
Epoch 240, Loss: 0.1392
